In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
BASE_PATH = '/home/marco/ifood-case'
DATA_RAW_PATH = BASE_PATH + '/data/raw/'

# profile

In [3]:
df_profile = pd.read_json(DATA_RAW_PATH + 'profile.json')

df_profile['age'] = df_profile['age'].astype(int)
df_profile = df_profile[df_profile['age'] <= 101]
df_profile['registered_on'] = pd.to_datetime(df_profile['registered_on'], format='%Y%m%d')
df_profile['gender'] = df_profile['gender'].str.upper().fillna('O')
df_profile['id'] = df_profile['id'].str.strip()
df_profile['credit_card_limit'] = df_profile['credit_card_limit'].astype(float)
df_profile = df_profile.rename(columns={'id': 'account_id'})

# offers

In [4]:
df_offers = pd.read_json(DATA_RAW_PATH + 'offers.json')

df_offers['min_value'] = df_offers['min_value'].astype(float)
df_offers['duration'] = df_offers['duration'].astype(int)
df_offers['id'] = df_offers['id'].str.strip()
df_offers['discount_value'] = df_offers['discount_value'].astype(float)

df_offers = df_offers.rename(columns={'id': 'offer_id'})

# transactions

In [5]:
df_transactions = pd.read_json(DATA_RAW_PATH + 'transactions.json')

df_transactions['account_id'] = df_transactions['account_id'].str.strip()
df_transactions['time_since_test_start'] = df_transactions['time_since_test_start'].astype(float)

value_expanded = df_transactions['value'].apply(pd.Series)
df_transactions_s = pd.concat([df_transactions.drop(columns='value'), value_expanded], axis=1)

In [6]:
df_transaction = df_transactions_s[df_transactions_s['event'] == 'transaction']
df_offer_received = df_transactions_s[df_transactions_s['event'] == 'offer received']
df_offer_viewed = df_transactions_s[df_transactions_s['event'] == 'offer viewed']
df_offer_completed = df_transactions_s[df_transactions_s['event'] == 'offer completed']

df_transaction = df_transaction[['account_id', 'amount', 'time_since_test_start']]
df_offer_received = df_offer_received[['account_id', 'offer id', 'time_since_test_start']]
df_offer_viewed = df_offer_viewed[['account_id', 'offer id', 'time_since_test_start']]
df_offer_completed = df_offer_completed[['account_id', 'offer_id', 'reward', 'time_since_test_start']]

df_transaction = df_transaction.rename(columns={
    'time_since_test_start': 'transaction_time',
})
df_offer_received = df_offer_received.rename(columns={
    'offer id': 'offer_id',
    'time_since_test_start': 'received_time',
})
df_offer_viewed = df_offer_viewed.rename(columns={
    'offer id': 'offer_id',
    'time_since_test_start': 'viewed_time',
})
df_offer_completed = df_offer_completed.rename(columns={
    'time_since_test_start': 'transaction_time',
})

# Sample profile

In [7]:
profile_id_s = '0610b486422d4921ae7d2bf64640c50b'

## Filters

In [8]:
df_transaction_s = df_transaction[df_transaction['account_id'] == profile_id_s]
df_transaction_s

,account_id,amount,transaction_time
20282,0610b486422d4921ae7d2bf64640c50b,21.51,0.75
49501,0610b486422d4921ae7d2bf64640c50b,32.28,6.00
227842,0610b486422d4921ae7d2bf64640c50b,23.22,22.00


In [9]:
df_offer_received_s = df_offer_received[df_offer_received['account_id'] == profile_id_s]
df_offer_received_s

,account_id,offer_id,received_time
150597,0610b486422d4921ae7d2bf64640c50b,9b98b8c7a33c4b65b9aebfe6a799e6d9,17.0
201571,0610b486422d4921ae7d2bf64640c50b,3f207df678b143eea3cee63160fa8bed,21.0


In [10]:
df_offer_viewed_s = df_offer_viewed[df_offer_viewed['account_id'] == profile_id_s]
df_offer_viewed_s

,account_id,offer_id,viewed_time


In [11]:
df_offer_completed_s = df_offer_completed[df_offer_completed['account_id'] == profile_id_s]
df_offer_completed_s

,account_id,offer_id,reward,transaction_time
227843,0610b486422d4921ae7d2bf64640c50b,9b98b8c7a33c4b65b9aebfe6a799e6d9,5.0,22.0


## Merge

In [12]:
df_transaction_m = (
    df_transaction_s
    .merge(
        df_offer_completed_s, 
        on=['account_id', 'transaction_time'], 
        how='left'
    )
    .merge(
        df_offer_received_s, 
        on=['account_id', 'offer_id'], 
        how='outer'
    )
    .merge(
        df_offer_viewed_s, 
        on=['account_id', 'offer_id'], 
        how='outer'
    )
    .merge(
        df_offers,
        on='offer_id',
        how='left'
    )
    .merge(
        df_profile,
        on='account_id',
        how='left'
    )
)

df_transaction_m['amount'] = df_transaction_m['amount'].fillna(0)
df_transaction_m['reward'] = df_transaction_m['reward'].fillna(0)

df_transaction_m['offer_conv_days'] = df_transaction_m['transaction_time'] - df_transaction_m['received_time']

df_transaction_m['actual_date'] = pd.to_datetime('2019-01-01')
df_transaction_m['registered_days'] = (df_transaction_m['actual_date'] - df_transaction_m['registered_on']).dt.days

df_transaction_m

,account_id,amount,transaction_time,offer_id,reward,received_time,viewed_time,channels,min_value,duration,offer_type,discount_value,age,registered_on,gender,credit_card_limit,offer_conv_days,actual_date,registered_days
0,0610b486422d4921ae7d2bf64640c50b,0.00,NaN,3f207df678b143eea3cee63160fa8bed,0.0,21.0,NaN,"[web, email, mobile]",0.0,4.0,informational,0.0,55,2017-07-15,F,112000.0,NaN,2019-01-01,535
1,0610b486422d4921ae7d2bf64640c50b,23.22,22.00,9b98b8c7a33c4b65b9aebfe6a799e6d9,5.0,17.0,NaN,"[web, email, mobile]",5.0,7.0,bogo,5.0,55,2017-07-15,F,112000.0,5.0,2019-01-01,535
2,0610b486422d4921ae7d2bf64640c50b,21.51,0.75,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55,2017-07-15,F,112000.0,NaN,2019-01-01,535
3,0610b486422d4921ae7d2bf64640c50b,32.28,6.00,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55,2017-07-15,F,112000.0,NaN,2019-01-01,535


## TimeLine

In [13]:
df_transaction_tl = (
    df_transaction
    .merge(
        df_offer_completed,
        on=['account_id', 'transaction_time'],
        how='left'
    )
)

df_transaction_tl = df_transaction_tl[[
    'account_id', 'offer_id', 'transaction_time'
]].rename(columns={
    'transaction_time': 'time'
})

df_transaction_tl['event'] = np.where(
    df_transaction_tl['offer_id'].notnull(),
    'offer_completed', 'transaction'
)

df_offer_received_tl = df_offer_received.rename(columns={
    'received_time': 'time'
})
df_offer_received_tl['event'] = 'offer_received'

df_timeline = pd.concat([df_transaction_tl, df_offer_received_tl])
df_timeline = df_timeline.drop_duplicates()
df_timeline = df_timeline.sort_values(['account_id', 'time'])

df_offers_tl = df_offers[['offer_id', 'duration', 'offer_type']]
df_timeline = df_timeline.merge(df_offers_tl, on='offer_id', how='left')
df_timeline['offer_type'] = df_timeline['offer_type'].fillna('no_offer')

In [14]:
df_timeline.groupby(['account_id', 'time']).count().query("event > 4")

,,offer_id,event,duration,offer_type
account_id,time,,,,
75bb371cf36d4a9186397a9866ed2fbe,24.0,5,5,5,5


In [15]:
df_timeline[df_timeline['account_id'] == '75bb371cf36d4a9186397a9866ed2fbe']

,account_id,offer_id,time,event,duration,offer_type
99521,75bb371cf36d4a9186397a9866ed2fbe,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,offer_received,7.0,bogo
99522,75bb371cf36d4a9186397a9866ed2fbe,f19421c1d4aa40978ebb69ca19b0e20d,7.0,offer_received,5.0,bogo
99523,75bb371cf36d4a9186397a9866ed2fbe,0b1e1539f2cc45b7b9fa7c272da2e1d7,14.0,offer_received,10.0,discount
99524,75bb371cf36d4a9186397a9866ed2fbe,2298d6c36e964ae4a3e7e9706d1fb8c2,17.0,offer_received,7.0,discount
99525,75bb371cf36d4a9186397a9866ed2fbe,4d5c57ea9a6940dd891ad53e9dbe8da0,21.0,offer_received,5.0,bogo
99526,75bb371cf36d4a9186397a9866ed2fbe,0b1e1539f2cc45b7b9fa7c272da2e1d7,24.0,offer_completed,10.0,discount
99527,75bb371cf36d4a9186397a9866ed2fbe,2298d6c36e964ae4a3e7e9706d1fb8c2,24.0,offer_completed,7.0,discount
99528,75bb371cf36d4a9186397a9866ed2fbe,4d5c57ea9a6940dd891ad53e9dbe8da0,24.0,offer_completed,5.0,bogo
99529,75bb371cf36d4a9186397a9866ed2fbe,fafdcd668e3743c1bb461111dcafc2a4,24.0,offer_completed,10.0,discount
99530,75bb371cf36d4a9186397a9866ed2fbe,fafdcd668e3743c1bb461111dcafc2a4,24.0,offer_received,10.0,discount


### Information offers

In [16]:
df_timeline_info = df_timeline[df_timeline['offer_type'].isin(['informational', 'no_offer'])]

In [17]:
df_timeline_info.groupby('account_id').count().sort_values('time')

,offer_id,time,event,duration,offer_type
account_id,,,,,
e2600c963f26487fa169839fff858ec0,0,1,1,0,1
66fcab7f93714bb88603096b1fdd72b1,0,1,1,0,1
66f09abc2d924907b5c49e01b1ad6d79,0,1,1,0,1
9b1f748c79ae43bc93c10499159341f8,0,1,1,0,1
0cc6e8553c844c02ab525bc466aa569b,1,1,1,1,1
...,...,...,...,...,...
b1f4ece7d49342628a9ed77aee2cde58,0,29,29,0,29
94de646f7b6041228ca7dec82adb97d2,0,30,30,0,30
4142f5e23db741b1af4be0287dc91c1c,1,30,30,1,30


In [18]:
df_timeline_info[df_timeline_info['account_id'] == '79d9d4f86aca4bed9290350fb43817c2']

,account_id,offer_id,time,event,duration,offer_type
102928,79d9d4f86aca4bed9290350fb43817c2,NaN,3.25,transaction,NaN,no_offer
102929,79d9d4f86aca4bed9290350fb43817c2,NaN,4.00,transaction,NaN,no_offer
102930,79d9d4f86aca4bed9290350fb43817c2,NaN,4.25,transaction,NaN,no_offer
102931,79d9d4f86aca4bed9290350fb43817c2,NaN,4.50,transaction,NaN,no_offer
102932,79d9d4f86aca4bed9290350fb43817c2,NaN,5.00,transaction,NaN,no_offer
102933,79d9d4f86aca4bed9290350fb43817c2,NaN,5.50,transaction,NaN,no_offer
102934,79d9d4f86aca4bed9290350fb43817c2,NaN,6.00,transaction,NaN,no_offer
102935,79d9d4f86aca4bed9290350fb43817c2,NaN,6.25,transaction,NaN,no_offer
102937,79d9d4f86aca4bed9290350fb43817c2,NaN,7.25,transaction,NaN,no_offer
102938,79d9d4f86aca4bed9290350fb43817c2,NaN,7.50,transaction,NaN,no_offer
